# Import

In [1]:
import os, re, pandas as pd
import numpy as np

# Extract result

In [2]:
result_pattern = re.compile(r'mse:(?P<mse>\d+.\d+), mae:(?P<mae>\d+.\d+)\n')
memory_pattern = re.compile(r'Allocated (?P<allocated>\d+.\d+) MB, Max allocated (?P<max_allocated>\d+.\d+) MB\n')
memory_pattern2 = re.compile(r'Allocated (?P<allocated>\d+.\d+) MB\n')

In [3]:
time_pattern = re.compile(r'Time per epoch: (?P<time>\d+.\d+) sec.\n')

In [12]:
def find_and_add(rows, root, lines):
    parameters = {
        'seq_len': 96,
        'pred_len': 24,
        'percent': 100,
    }

    splits = root.split('\\')
    iter_no = int(splits[-1])
    dataset = splits[-3]
    
    if dataset == 'Financial_Aid':
        return
    model_folder = splits[-2]
    model = model_folder.split('_')[0]

    values = model_folder.split('_')[2::2]
    for i, item in enumerate(model_folder.split('_')[1::2]):
        if item == 'pl':
            parameters['pred_len'] = int(values[i])
        elif item == 'sl':
            parameters['seq_len'] = int(values[i])
        elif item == 'p':
            parameters['percent'] = int(values[i])

    result_found = memory_found = time_found = False

    for line in lines[::-1]:
        # search and extract numbers after mse and mae if the line starts with test scaled 
        if line.startswith('test scaled -- '):
            results = result_pattern.search(line).groupdict()
            mae = float(results['mae'])
            mse = float(results['mse'])
            print(f'MAE {mae}, MSE {mse}')
            result_found = True
            
        elif line.startswith('Memory usage: '):
            memory = memory_pattern.search(line)
            if memory is None:
                memory = memory_pattern2.search(line)
                max_allocated = None
            else:
                memory = memory.groupdict()
                max_allocated = float(memory['max_allocated'])
            
            allocated = float(memory['allocated'])
            print(f'Allocated {allocated} MB, Max allocated {max_allocated} MB.')
            memory_found = True
        elif line.startswith('Time per epoch: '):
            time_result = time_pattern.search(line).groupdict()
            time_spent = float(time_result['time'])
            print(f'Time spent {time_spent} sec.')
            
            time_found = True
            
        # stop searching if both result and memory are found
        if result_found and memory_found and time_found:
            break
            
    if result_found and memory_found and time_found:
        rows['dataset'].append(dataset)
        rows['model'].append(model)
        rows['iteration'].append(iter_no)
        rows['mae'].append(mae)
        rows['mse'].append(mse)
        rows['seq_len'].append(parameters['seq_len'])
        rows['pred_len'].append(parameters['pred_len'])
        rows['percent'].append(parameters['percent'])
        
        rows['memory'].append(allocated)
        rows['time'].append(time_spent)

In [ ]:
rows = {
    col:[] for col in [
        'dataset', 'model', 'iteration', 'mae', 
        'mse', 'seq_len', 'pred_len', 'percent',
        'memory', 'time'
    ]
}

for root, dirs, files in os.walk(u"."):
    path = root.split(os.sep)
    # print((len(path) - 1) * '---', os.path.basename(root))
    for file in files:
        # print(len(path) * '---', file)
        if file == 'results.txt':
            print(root, ' ', file)
            
            with open(os.path.join(root, file), 'r') as f:
                lines = f.readlines()
                find_and_add(rows, root, lines)

In [14]:
rows_df = pd.DataFrame(rows)
rows_df.to_csv('results2.csv', index=False)

In [15]:
rows_df.fillna('mean', inplace=True)

In [16]:
metrics = ['mae', 'mse', 'memory', 'time']
groupby_columns = [col for col in rows_df.columns if col not in ['iteration']+metrics]
rows_df = rows_df.groupby(groupby_columns)[metrics].mean().reset_index()

In [17]:
models = ['DLinear', 'PatchTST', 'TimesNet', 'TimeMixer', 'iTransformer', 'TimeLLM', 'CALF', 'OFA']
datasets = ['Exchange_Rate_Report', 'SPX500', 'MSFT', 'Apple','Natural_Gas','Crude_Oil', 'Gold']

# RQ1

In [18]:
df = rows_df[(rows_df['pred_len']==24) & (rows_df['percent']==100)]

In [19]:
def get_mins(mses, maes):
    maes = np.array(maes)
    mses = np.array(mses)
    
    min_mae = np.nanargmin(maes)
    min_mse = np.nanargmin(mses)
    
    prev_value = maes[min_mae]
    maes[min_mae] = np.inf
    min_mae_2nd = np.nanargmin(maes)
    maes[min_mae] = prev_value
    
    prev_value = mses[min_mse]
    mses[min_mse] = np.inf
    min_mse_2nd = np.nanargmin(mses)
    mses[min_mse] = prev_value
    
    return min_mae, min_mse, min_mae_2nd, min_mse_2nd

In [20]:
for dataset in datasets:
    print(f'{dataset} ', end='')
    maes, mses = [], []
    for model in models:
        result = df[
            (df['dataset'] == dataset) & (df['model'] == model)
        ][['mse', 'mae']].values
        
        if len(result) == 0:
            # print(f' & & ', end='')
            maes.append(np.nan)
            mses.append(np.nan)
            continue
            
        mse, mae = result[0][0], result[0][1]
        # print(f' & {mse:.2f} & {mae:.2f}', end='')
        maes.append(mae)
        mses.append(mse)
        
    min_mae, min_mse, min_mae_2nd, min_mse_2nd = get_mins(mses=mses, maes=maes)
    
    for i in range(len(models)):
        if i == min_mse:
            print(r' & \textcolor{red}{\textbf{' + f'{mses[i]:.2f} '+r'}}', end='')
        elif i == min_mse_2nd:
            print(r' & \textcolor{blue}{\underline{' + f'{mses[i]:.2f} '+r'}}', end='')
        else:
            print(f' & {mses[i]:.2f} ', end='')
            
        if i == min_mae:
            print(r' & \textcolor{red}{\textbf{' + f'{maes[i]:.2f} '+r'}}', end='')
        elif i == min_mae_2nd:
            print(r' & \textcolor{blue}{\underline{' + f'{maes[i]:.2f} '+r'}}', end='')
        else:
            print(f' & {maes[i]:.2f} ', end='')
        
    print(f' \\\\ \\hline')

Exchange_Rate_Report  & \textcolor{red}{\textbf{0.92 }} & \textcolor{red}{\textbf{0.70 }} & 0.94  & \textcolor{blue}{\underline{0.71 }} & 1.71  & 0.99  & 0.95  & 0.72  & 1.06  & 0.77  & \textcolor{blue}{\underline{0.93 }} & 0.73  & 1.14  & 0.81  & 0.97  & 0.72  \\ \hline
SPX500  & 1.10  & 0.81  & \textcolor{blue}{\underline{0.99 }} & 0.79  & 1.18  & 0.84  & 1.00  & \textcolor{blue}{\underline{0.75 }} & 1.27  & 0.87  & 1.10  & 0.81  & 1.16  & 0.84  & \textcolor{red}{\textbf{0.79 }} & \textcolor{red}{\textbf{0.68 }} \\ \hline
MSFT  & 1.89  & 0.96  & \textcolor{red}{\textbf{1.55 }} & \textcolor{blue}{\underline{0.87 }} & 1.60  & 0.92  & \textcolor{blue}{\underline{1.55 }} & \textcolor{red}{\textbf{0.86 }} & 1.63  & 0.91  & 2.03  & 1.01  & 1.94  & 1.00  & 1.90  & 0.93  \\ \hline
Apple  & \textcolor{red}{\textbf{1.58 }} & \textcolor{red}{\textbf{0.96 }} & 2.03  & 1.06  & 1.75  & \textcolor{blue}{\underline{1.01 }} & 1.91  & 1.06  & 1.94  & 1.10  & \textcolor{blue}{\underline{1.74 }} & 1.02 

# RQ2

In [21]:
df = rows_df[(rows_df['pred_len']==24) & (rows_df['percent']==10)]

In [22]:
for dataset in datasets:
    print(f'{dataset} ', end='')
    maes, mses = [], []
    for model in models:
        result = df[
            (df['dataset'] == dataset) & (df['model'] == model)
        ][['mse', 'mae']].values
        
        if len(result) == 0:
            # print(f' & & ', end='')
            maes.append(np.nan)
            mses.append(np.nan)
            continue
            
        mse, mae = result[0][0], result[0][1]
        # print(f' & {mse:.2f} & {mae:.2f}', end='')
        maes.append(mae)
        mses.append(mse)
        
    min_mae, min_mse, min_mae_2nd, min_mse_2nd = get_mins(mses=mses, maes=maes)
    
    for i in range(len(models)):
        if i == min_mse:
            print(r' & \textcolor{red}{\textbf{' + f'{mses[i]:.2f} '+r'}}', end='')
        elif i == min_mse_2nd:
            print(r' & \textcolor{blue}{\underline{' + f'{mses[i]:.2f} '+r'}}', end='')
        else:
            print(f' & {mses[i]:.2f} ', end='')
            
        if i == min_mae:
            print(r' & \textcolor{red}{\textbf{' + f'{maes[i]:.2f} '+r'}}', end='')
        elif i == min_mae_2nd:
            print(r' & \textcolor{blue}{\underline{' + f'{maes[i]:.2f} '+r'}}', end='')
        else:
            print(f' & {maes[i]:.2f} ', end='')
        
    print(f' \\\\ \\hline')

Exchange_Rate_Report  & 1.48  & 0.92  & 1.28  & 0.85  & 2.87  & 1.34  & 1.29  & 0.84  & 1.65  & 0.96  & \textcolor{red}{\textbf{1.20 }} & \textcolor{red}{\textbf{0.81 }} & 1.48  & 0.92  & \textcolor{blue}{\underline{1.22 }} & \textcolor{blue}{\underline{0.81 }} \\ \hline
SPX500  & 2.08  & 1.14  & \textcolor{blue}{\underline{2.03 }} & \textcolor{red}{\textbf{1.14 }} & 2.43  & 1.18  & 2.49  & 1.25  & 2.19  & 1.19  & \textcolor{red}{\textbf{1.94 }} & \textcolor{blue}{\underline{1.14 }} & 2.37  & 1.19  & 3.07  & 1.40  \\ \hline
MSFT  & 2.51  & 1.10  & \textcolor{red}{\textbf{2.13 }} & \textcolor{red}{\textbf{1.06 }} & 3.55  & 1.43  & 2.85  & 1.18  & 2.49  & 1.15  & \textcolor{blue}{\underline{2.40 }} & \textcolor{blue}{\underline{1.10 }} & 2.97  & 1.27  & 3.40  & 1.37  \\ \hline
Apple  & 2.78  & 1.30  & \textcolor{blue}{\underline{2.36 }} & \textcolor{red}{\textbf{1.21 }} & 3.22  & 1.41  & 3.44  & 1.46  & 3.05  & 1.39  & 3.00  & 1.36  & \textcolor{red}{\textbf{2.33 }} & \textcolor{blue}{\u

# RQ3

# RQ4

In [23]:
for dataset in datasets:
    for horizon in [48, 96]:
        df = rows_df[(rows_df['pred_len']==horizon) & (rows_df['percent']==100)]
        print(f'{dataset} & {horizon}', end='')
        maes, mses = [], []
        for model in models:
            result = df[
                (df['dataset'] == dataset) & (df['model'] == model)
            ][['mse', 'mae']].values
            
            if len(result) == 0:
                # print(f' & & ', end='')
                maes.append(np.nan)
                mses.append(np.nan)
                continue
                
            mse, mae = result[0][0], result[0][1]
            # print(f' & {mse:.2f} & {mae:.2f}', end='')
            maes.append(mae)
            mses.append(mse)
            
        min_mae, min_mse, min_mae_2nd, min_mse_2nd = get_mins(mses=mses, maes=maes)
        
        for i in range(len(models)):
            if i == min_mse:
                print(r' & \textcolor{red}{\textbf{' + f'{mses[i]:.2f} '+r'}}', end='')
            elif i == min_mse_2nd:
                print(r' & \textcolor{blue}{\underline{' + f'{mses[i]:.2f} '+r'}}', end='')
            else:
                print(f' & {mses[i]:.2f} ', end='')
                
            if i == min_mae:
                print(r' & \textcolor{red}{\textbf{' + f'{maes[i]:.2f} '+r'}}', end='')
            elif i == min_mae_2nd:
                print(r' & \textcolor{blue}{\underline{' + f'{maes[i]:.2f} '+r'}}', end='')
            else:
                print(f' & {maes[i]:.2f} ', end='')
            
        print(f' \\\\ \\hline')

Exchange_Rate_Report & 48 & \textcolor{red}{\textbf{1.42 }} & \textcolor{red}{\textbf{0.88 }} & \textcolor{blue}{\underline{1.49 }} & \textcolor{blue}{\underline{0.91 }} & 2.65  & 1.29  & 1.50  & 0.91  & 1.80  & 1.01  & 1.58  & 0.96  & 1.86  & 1.05  & 1.52  & 0.92  \\ \hline
Exchange_Rate_Report & 96 & \textcolor{red}{\textbf{1.73 }} & \textcolor{blue}{\underline{0.98 }} & \textcolor{blue}{\underline{1.77 }} & \textcolor{red}{\textbf{0.96 }} & 3.93  & 1.63  & 1.82  & 1.02  & 1.88  & 1.03  & 1.88  & 1.05  & 2.17  & 1.14  & 1.98  & 1.07  \\ \hline
SPX500 & 48 & 2.24  & 1.15  & 2.05  & 1.08  & 2.30  & 1.13  & \textcolor{red}{\textbf{1.51 }} & \textcolor{red}{\textbf{0.93 }} & 2.62  & 1.26  & 2.10  & 1.11  & 1.98  & 1.08  & \textcolor{blue}{\underline{1.63 }} & \textcolor{blue}{\underline{0.94 }} \\ \hline
SPX500 & 96 & 9.84  & 2.20  & 5.31  & \textcolor{blue}{\underline{1.65 }} & 6.17  & 1.78  & 5.72  & 1.71  & 6.54  & 1.87  & \textcolor{blue}{\underline{5.25 }} & 1.67  & 5.93  & 1.73  & 

# RQ5

In [24]:
df = rows_df[(rows_df['pred_len']==24) & (rows_df['percent']==100)]

In [25]:
for dataset in datasets:
    print(f'{dataset} ', end='')
    times, memories = [], []
    for model in models:
        result = df[
            (df['dataset'] == dataset) & (df['model'] == model)
        ][['time', 'memory']].values
        
        if len(result) == 0:
            # print(f' & & ', end='')
            times.append(np.nan)
            memories.append(np.nan)
            continue
            
        time_spent, memory = result[0][0], result[0][1]
        # print(f' & {mse:.2f} & {mae:.2f}', end='')
        times.append(time_spent)
        memories.append(memory)
        
    min_memory, min_time, min_memory_2nd, min_time_2nd = get_mins(mses=times, maes=memories)
    
    for i in range(len(models)):
        if i == min_time:
            print(r' & \textcolor{red}{\textbf{' + f'{times[i]:.1f} '+r'}}', end='')
        elif i == min_time_2nd:
            print(r' & \textcolor{blue}{\underline{' + f'{times[i]:.1f} '+r'}}', end='')
        else:
            print(f' & {times[i]:.1f} ', end='')
            
        if i == min_memory:
            print(r' & \textcolor{red}{\textbf{' + f'{memories[i]:.0f} '+r'}}', end='')
        elif i == min_memory_2nd:
            print(r' & \textcolor{blue}{\underline{' + f'{memories[i]:.0f} '+r'}}', end='')
        else:
            print(f' & {memories[i]:.0f} ', end='')
        
    print(f' \\\\ \\hline')
    print()

Exchange_Rate_Report  & \textcolor{red}{\textbf{1.2 }} & \textcolor{red}{\textbf{16 }} & \textcolor{blue}{\underline{1.7 }} & 19  & 51.2  & 167  & 4.6  & 18  & 1.7  & \textcolor{blue}{\underline{18 }} & 60.6  & 1127  & 11.7  & 1462  & 7.7  & 350  \\ \hline

SPX500  & \textcolor{red}{\textbf{2.0 }} & \textcolor{red}{\textbf{16 }} & 3.4  & 19  & 19.5  & 170  & 4.3  & 18  & \textcolor{blue}{\underline{2.8 }} & \textcolor{blue}{\underline{18 }} & 37.5  & 1127  & 12.1  & 1473  & 5.9  & 349  \\ \hline

MSFT  & \textcolor{red}{\textbf{2.6 }} & \textcolor{red}{\textbf{16 }} & 3.3  & 19  & 17.4  & 170  & 5.0  & 18  & \textcolor{blue}{\underline{3.3 }} & \textcolor{blue}{\underline{18 }} & 47.5  & 1127  & 11.6  & 1470  & 6.4  & 349  \\ \hline

Apple  & \textcolor{red}{\textbf{2.2 }} & \textcolor{red}{\textbf{16 }} & 4.1  & 19  & 24.0  & 167  & 4.7  & 18  & \textcolor{blue}{\underline{3.3 }} & \textcolor{blue}{\underline{18 }} & 53.5  & 1127  & 13.1  & 1470  & 5.4  & 349  \\ \hline

Natural_Gas  